* In continuation with post: [tokenize-train-data-using-bert-tokenizer](https://www.kaggle.com/harveenchadha/tokenize-train-data-using-bert-tokenizer/notebook)

* After you save the tokenizers output in npy format, following code could be implemented in order to reloading as torch.data.Dataset and then passing it to a dataloader.

* UseCase:-
Large training data so to avoid on the fly tokenizatio, we save tokenized data to disk and load it using below code.


In [ ]:
train_token = np.load('token_train.npy', allow_pickle=True)
#the BERT tokenization is saved as token_train.npy, see the post to find how it is saved
train_token=train_token.tolist()

In [ ]:
print(token_train.keys())
# O/P - dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])

In [ ]:
class MyDataset(Dataset):
    
    def __init__(self, dict_,  labels =False):
        self.dict = dict_
        self.labels = labels
    def __getitem__(self, ids):

        return {
            'input_ids' : torch.tensor(self.dict['input_ids'][ids], dtype=torch.long),
            'token_type_ids' : torch.tensor(self.dict['token_type_ids'][ids], dtype=torch.long),
            'attention_mask' : torch.tensor(self.dict['attention_mask'][ids], dtype=torch.long),
            'labels' : torch.tensor(self.dict['label'][ids], dtype=torch.long)
        }
    
    def __len__(self):
        return len(self.dict['label'])

After fetching a list of samples using the indices[ids here] from sampler, the function passed as the collate_fn argument is used to collate lists of samples into batches

In [ ]:
def collate(batch):
    ips = [item['input_ids'] for item in batch]
    ttypes = [item['token_type_ids'] for item in batch]
    attn = [item['attention_mask'] for item in batch]
    lb = [item['labels'] for item in batch]

    return {
               'token_type_ids': torch.stack(ttypes),
               'input_ids': torch.stack(ips),
               'attention_mask' : torch.stack(attn),
               'labels': torch.stack(lb) 
            }

In [ ]:
dataset = MyDataset(train_token,)
train_dataloader =  DataLoader(dataset,batch_size=128, collate_fn= collate, shuffle=True)